In [ ]:
'''
Breaks the large raw tweet data file into manageable sections

'''

import pandas as pd

chunk_size = 10000
batch_no = 1

print("Beginning to parse tweets...")

for chunk in pd.read_csv("01-03_covid_data.csv", chunksize = chunk_size, error_bad_lines=False):
        chunk.to_csv("covid_data" + str(batch_no) + ".csv", index = False)
        batch_no += 1

print("Finished parsing tweets.")

In [ ]:
'''
Gensim is a topic-modelling machine learning program
This should help sort the sentiments into respective topics for better analysis
'''

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from gensim import corpora
np.random.seed(2020)
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    return SnowballStemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result
import csv
import glob

sentimentDict = {}
tweet_text = []
path = "./covid_data*"

print("Beginning to create corpus...")
#I put the smaller chunked tweet files in to a directory called revisedCovidData
for filename in glob.glob(path):
    #Iterate through this directory, reading each file
    with open(filename, 'r', encoding="utf-8") as rawTweets:
        #open as CSV iterator
        readCSV = csv.reader(rawTweets)
        #Iterate through individual tweets
        for line in readCSV:
            #if line[9] != "Null" or "us_state":
                #calls text of each tweet as a TextBlob object
            text = line[1]
                #line[9] = state; if this state is already in the dictionary, the sentiment gets averaged
            tweet_text.append(text)
        break

data_lemmatized = lemmatize_stemming(tweet_text) #####THIS LINE HAS AN ERROR#####
preprocessed_tweets = preprocess(tweet_text)
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(tweet) for tweet in tweet_text]

#denote finished. These are large files, it takes a while
print("Finsished texts LDA.")

In [ ]:
'''
Build LDA model
'''
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word=id2word,
                                           num_topics = 5,
                                           random_state = 100 
                                           update_every=1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha='auto',
                                           per_word_topics = True)

In [ ]:
#OPTIONAL: measure how good the model is. Lower is better
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

#compute coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts = data_lemmatized, dictionary = id2word, coherence='c_V')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
'''
Creates a dictionary mapping the value of "state" in each tweet with the average sentiment

senitiment.polarity: denotes the sentiment of a text, float from -1 (completely negative) to 1 (completely positive) 

STILL NEEDS: sort by topic and sort into sliding window by 1 week intervals

'''
import csv
import glob

outfile = "TweetTextOnly.txt"
tweetList = []
path = "./covid_data*"

print("Beginning to assign sentiments...")
#I put the smaller chunked tweet files in to a directory called revisedCovidData
for filename in glob.glob(path):
    #Iterate through this directory, reading each file
    with open(filename, 'r', encoding="utf-8") as rawTweets:
        #open as CSV iterator
        readCSV = csv.reader(rawTweets)
        #Iterate through individual tweets
        for line in readCSV:
            if line[9] != "Null" or "us_state":
                #calls text of each tweet as a TextBlob object
                text = textblob.TextBlob(line[1])
                #line[9] = state; if this state is already in the dictionary, the sentiment gets averaged
                if line[9] in sentimentDict:
                    sentimentDict[line[9]] = (sentimentDict[line[9]][1] +  (text.sentiment.polarity))/2
                    #sentimentDict[line[9]].append(text.sentiment.polarity) #<= if wanting a list instead of an average
                else:
                    #it state not already accounted for, saves it as a key with the relevant sentiment
                    sentimentDict.update({line[9]: (text.sentiment.polarity)})
                
                
    break

#denote finished. These are large files, it takes a while
print("Finished assigning sentiments.")

In [ ]:
"""
(Optional) writes dictionary to CSV file with rows of state, sentiment
"""

print("writing sentiments to file...")
with open("twitter_sentiments.csv", "w") as outFile:
    writer = csv.writer(outFile)
    writer.writerow(["State", "Avg. Sentiment"])
    for key, value in sentimentDict.items():
        writer.writerow([key, value])
print("finsished writing to file")